В данном ноутбуке приведены этапы исследования способов ранжирования документов по текстовой релевантности.

При этом максимальный результат получился таким: 0.72044 / 0.74728 (private score / public score).
Вы можете достичь его, последовательно выполняя ячейки разделов: "Подготовка данных", "Новый индекс: с разделением документа на части", "BM25F с расширениями запросов".

## Подготовка данных

In [1]:
import pandas as pd

urls = pd.read_csv('urls.numerate.txt', sep="	", header=None)
urls.columns = ['id', 'url']

queries = pd.read_csv('queries.numerate2.txt', sep="	", header=None)
queries.columns = ['id', 'query']

sample = pd.read_csv('sample.technosphere.ir1.textrelevance.submission.txt', sep=",")

urls_num = urls.shape[0]
print(urls_num)

queries_num = queries.shape[0]
print(queries_num)

doc_ids = {}
for i in range(queries_num):
    doc_ids[i] = list(sample[sample.QueryId == i+1]['DocumentId'])

urls_to_query = {}
for i in range(queries_num):
    urls_list = []
    for j in doc_ids[i]:
        urls_list.append(urls[urls.id == j]['url'][j-1])        
    urls_to_query[queries[queries.id == i+1]['query'][i]] = urls_list

test_url = urls_to_query[queries['query'][7]][15]
print(test_url)

38114
399
nerzon.ru/article/igrat_v_majjnkraft_bez_registracii.html


In [2]:
from html.parser import HTMLParser
import re
import os
import csv

files = []
for i in range(8):
    directory = 'content/content/'+str(i+1)
    files_list = os.listdir(directory)
    for ind in range(len(files_list)):
        files_list[ind] = directory + '/' + files_list[ind]
    files.append(files_list)

all_files = []
for filenames in files:
    for filename in filenames:
        all_files.append(filename)

"""docs_id = []
n = 0
for file in all_files:
    n += 1
    if n%5000 == 0:
        print(n)
    f = open(file, 'r', encoding='utf-8', errors='ignore').readline()
    url = re.sub('\n', '', f)
    docs_id.append(int(urls[urls.url == url].id))
    
with open('docs_id.txt', 'w', encoding='utf-8', errors='ignore', newline = '') as out:
    writer = csv.writer(out)
    for one_id in docs_id:
        writer.writerow([one_id])
    """

"docs_id = []\nn = 0\nfor file in all_files:\n    n += 1\n    if n%5000 == 0:\n        print(n)\n    f = open(file, 'r', encoding='utf-8', errors='ignore').readline()\n    url = re.sub('\n', '', f)\n    docs_id.append(int(urls[urls.url == url].id))\n    \nwith open('docs_id.txt', 'w', encoding='utf-8', errors='ignore', newline = '') as out:\n    writer = csv.writer(out)\n    for one_id in docs_id:\n        writer.writerow([one_id])\n    "

In [3]:
file = pd.read_csv('docs_id.txt', header=None)
docs_id = list(file[0])

docs_to_query = {}
for i in range(queries_num):
    docs_list = []
    for j in doc_ids[i]:
        docs_list.append(all_files[docs_id.index(j)])        
    docs_to_query[queries[queries.id == i+1]['query'][i]] = docs_list

In [4]:
stopwords = pd.read_csv('stop_words_ru.txt', header=None)
stopwords.columns = ['words']

print('как' in list(stopwords.words))

def to_utf8(text):
    if isinstance(text, str): text = text.encode('utf8')
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, str): text = text.decode('utf8')
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

True


In [11]:
urls_count = {url: 0 for url in urls['url']}
urls_heads = {url: url[:url.find('/')] for url in urls['url']}

In [169]:
def calc_links(doc):
    soup = BeautifulSoup(doc, 'html.parser')
    url = re.sub('\n', '', doc)
    ind = url.find('/')
    head_url = url[:ind]
    for link in soup.find_all("a"):
        if link.has_attr('href'):
            cur_link = link['href']
            if (cur_link[:4] != 'http') or (head_url in cur_link):
                continue
            else:
                for url, head_url in urls_heads.items():
                    if head_url in cur_link:
                        urls_count[url] += 1

In [170]:
n = 0
for file in all_files:
    n += 1
    if n%1000 == 0:
        print(n)
    f = open(file, 'r', encoding='utf-8', errors='ignore').read()
    calc_links(f)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


## Вторая попытка построить индекс (включены английские слова и цифры)

In [112]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = str()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = str()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            word = get_lemmatizer().parse(word)[0].normal_form
            if word not in list(stopwords.words):
                PYMORPHY_CACHE[word_hash] = word
                yield PYMORPHY_CACHE[word_hash]
                
from bs4 import BeautifulSoup

def html2text_bs_visible(raw_html):
    soup = BeautifulSoup(raw_html, 'html.parser')
    [s.extract() for s in soup(['style', 'script', 'head', 'title', 'meta', '[document]'])]
    return soup.get_text()

In [113]:
def html2word(raw_html, to_text=html2text_bs_visible, tokenizer=pymorphy_tokenizer):
    return tokenizer(to_text(raw_html).lower())

In [33]:
def process_files(files):
    file_to_terms = {}
    with open('index1.txt', 'w', encoding='utf-8', errors='ignore') as fout:
        writer = csv.writer(fout)
        for filenames in files:
            print("***")
            for file in filenames:
                PYMORPHY_CACHE = {}
                f = open(file, 'r', encoding='utf-8', errors='ignore').read()
                items = list(html2word(f))
                print(len(items))
                file_to_terms[file] = items
                writer.writerow([file, items])
    return file_to_terms

In [152]:
k = 300
for query in queries['query'][301:]:
    k += 1
    print('************************************************')
    print(k)
    index1[query] = process_files(query)

************************************************
301
[1, 0]
[2, 1]
[3, 9]
[4, 5]
[5, 41]
[6, 41]
[7, 14]
[8, 5]
[9, 8]
[10, 6]
[11, 4]
[12, 2]
[13, 8]
[14, 82]
[15, 174]
[16, 3]
[17, 5]
[18, 15]
[19, 58]
[20, 95]
[21, 15]
[22, 3]
[23, 13]
[24, 1]
[25, 15]
[26, 22]
[27, 6]
[28, 75]
[29, 17]
[30, 19]
[31, 6]
[32, 6]
[33, 9]
[34, 21]
[35, 73]
[36, 41]
[37, 7]
[38, 6]
[39, 10]
[40, 29]
[41, 3]
[42, 7]
[43, 11]
[44, 5]
[45, 2125]
[46, 65]
[47, 767]
[48, 78]
[49, 38]
[50, 13]
[51, 84]
[52, 15]
[53, 19]
[54, 1]
[55, 23]
[56, 1]
[57, 1]
[58, 11]
[59, 6]
[60, 5]
[61, 3]
[62, 44]
[63, 20]
[64, 7]
[65, 2]
[66, 200]
[67, 9]
[68, 2]
[69, 27]
[70, 13]
[71, 2]
[72, 5]
[73, 3]
[74, 15]
[75, 1]
[76, 4]
[77, 37]
[78, 6]
[79, 11]
[80, 40]
[81, 9]
[82, 1]
[83, 11]
[84, 0]
[85, 8]
[86, 12]
[87, 23]
[88, 0]
[89, 0]
[90, 0]
[91, 1]
[92, 56]
[93, 11]
[94, 20]
[95, 13]
[96, 16]
[97, 3]
[98, 142]
[99, 0]
[100, 43]
[101, 0]
[102, 2]
************************************************
302
[1, 246]
[2, 297]
[3, 75]
[

[6, 5]
[7, 4]
[8, 50]
[9, 111]
[10, 1272]
[11, 53]
[12, 67]
[13, 11]
[14, 109]
[15, 33]
[16, 8]
[17, 39]
[18, 83]
[19, 77]
[20, 108]
[21, 6]
[22, 16]
[23, 2]
[24, 36]
[25, 76]
[26, 30]
[27, 7]
[28, 38]
[29, 19]
[30, 23]
[31, 195]
[32, 17]
[33, 39]
[34, 10]
[35, 46]
[36, 59]
[37, 49]
[38, 70]
[39, 3]
[40, 94]
[41, 70]
[42, 131]
[43, 5]
[44, 36]
[45, 8]
[46, 8]
[47, 77]
[48, 9]
[49, 13]
[50, 16]
[51, 24]
[52, 60]
[53, 23]
[54, 50]
[55, 3]
[56, 74]
[57, 37]
[58, 2]
[59, 49]
[60, 37]
[61, 7]
[62, 58]
[63, 12]
[64, 10]
[65, 26]
[66, 5]
[67, 3]
[68, 6]
[69, 85]
[70, 4]
[71, 8]
[72, 102]
[73, 4]
[74, 0]
[75, 64]
[76, 11]
[77, 24]
[78, 308]
[79, 21]
[80, 101]
[81, 40]
[82, 158]
[83, 54]
[84, 0]
[85, 81]
[86, 44]
[87, 17]
[88, 41]
[89, 49]
[90, 14]
[91, 9]
[92, 93]
[93, 8]
[94, 6]
[95, 7]
[96, 36]
[97, 49]
[98, 69]
[99, 41]
************************************************
311
[1, 25]
[2, 5]
[3, 3]
[4, 40]
[5, 28]
[6, 0]
[7, 0]
[8, 0]
[9, 1]
[10, 4]
[11, 21]
[12, 5]
[13, 4]
[14, 0]
[15, 6]
[16, 

[101, 611]
[102, 270]
************************************************
319
[1, 34]
[2, 77]
[3, 121]
[4, 54]
[5, 7]
[6, 3]
[7, 8]
[8, 103]
[9, 0]
[10, 23]
[11, 33]
[12, 31]
[13, 4]
[14, 10]
[15, 0]
[16, 6]
[17, 2]
[18, 22]
[19, 8]
[20, 38]
[21, 7]
[22, 4]
[23, 18]
[24, 17]
[25, 80]
[26, 76]
[27, 22]
[28, 2]
[29, 5]
[30, 10]
[31, 11]
[32, 43]
[33, 5]
[34, 7]
[35, 5]
[36, 14]
[37, 121]
[38, 2]
[39, 3]
[40, 28]
[41, 14]
[42, 10]
[43, 85]
[44, 11]
[45, 3]
[46, 1]
[47, 28]
[48, 0]
[49, 0]
[50, 5384]
[51, 17]
[52, 7]
[53, 6]
[54, 45]
[55, 12]
[56, 21]
[57, 6]
[58, 30]
[59, 11]
[60, 11]
[61, 10]
[62, 16]
[63, 9]
[64, 2]
[65, 8]
[66, 2]
[67, 6]
[68, 0]
[69, 4]
[70, 6]
[71, 0]
[72, 3]
[73, 37]
[74, 79]
[75, 9]
[76, 0]
[77, 0]
[78, 0]
[79, 1]
[80, 12]
[81, 3]
[82, 27]
[83, 7]
[84, 43]
[85, 14]
[86, 29]
[87, 0]
[88, 0]
[89, 9]
[90, 2]
[91, 0]
[92, 33]
[93, 8]
[94, 10]
[95, 29]
[96, 61]
[97, 12]
[98, 70]
[99, 11]
[100, 41]
[101, 19]
[102, 32]
[103, 5]
***********************************************

[33, 50]
[34, 27]
[35, 36]
[36, 0]
[37, 23]
[38, 5]
[39, 2]
[40, 49]
[41, 268]
[42, 20]
[43, 1]
[44, 11]
[45, 38]
[46, 7]
[47, 8]
[48, 4]
[49, 6]
[50, 61]
[51, 9]
[52, 9]
[53, 26]
[54, 728]
[55, 102]
[56, 5]
[57, 9]
[58, 5]
[59, 8]
[60, 0]
[61, 5]
[62, 6]
[63, 14]
************************************************
328
[1, 0]
[2, 140]
[3, 9]
[4, 41]
[5, 2]
[6, 46]
[7, 620]
[8, 242]
[9, 56]
[10, 14]
[11, 928]
[12, 247]
[13, 189]
[14, 33]
[15, 268]
[16, 311]
[17, 0]
[18, 581]
[19, 21]
[20, 5]
[21, 103]
[22, 5]
[23, 1004]
[24, 247]
[25, 96]
[26, 64]
[27, 201]
[28, 0]
[29, 237]
[30, 224]
[31, 358]
[32, 40]
[33, 6]
[34, 340]
[35, 19]
[36, 86]
[37, 5]
[38, 61]
[39, 0]
[40, 0]
[41, 326]
[42, 527]
[43, 27]
[44, 77]
[45, 224]
[46, 17]
[47, 30]
[48, 0]
[49, 33]
[50, 115]
[51, 194]
[52, 260]
[53, 662]
[54, 58]
[55, 0]
[56, 168]
[57, 0]
[58, 442]
[59, 530]
[60, 55]
[61, 82]
[62, 10]
[63, 317]
[64, 429]
[65, 187]
[66, 55]
[67, 3386]
[68, 147]
[69, 59]
[70, 522]
[71, 257]
[72, 36]
[73, 46]
[74, 159]
[7

[9, 8]
[10, 16]
[11, 43]
[12, 2]
[13, 17]
[14, 5]
[15, 5]
[16, 11]
[17, 5]
[18, 4]
[19, 3]
[20, 6]
[21, 17]
[22, 65]
[23, 7]
[24, 1]
[25, 4]
[26, 19]
[27, 3]
[28, 64]
[29, 3]
[30, 1]
[31, 5]
[32, 12]
[33, 10]
[34, 9]
[35, 17]
[36, 6]
[37, 13]
[38, 13]
[39, 5]
[40, 8]
[41, 5]
[42, 16]
[43, 5]
[44, 3]
[45, 11]
[46, 42]
[47, 4]
[48, 26]
[49, 1]
[50, 1]
[51, 357]
[52, 8]
[53, 20]
[54, 17]
[55, 1]
[56, 29]
[57, 6]
[58, 58]
[59, 14]
[60, 25]
[61, 26]
[62, 14]
[63, 34]
[64, 4]
[65, 2]
[66, 17]
[67, 27]
[68, 25]
[69, 116]
[70, 1]
[71, 11]
[72, 3]
[73, 8]
[74, 66]
[75, 3]
[76, 9]
[77, 64]
[78, 7]
[79, 19]
[80, 10]
[81, 11]
[82, 4]
[83, 16]
[84, 6]
[85, 1]
[86, 43]
[87, 4]
[88, 11]
[89, 8]
[90, 8]
[91, 9]
[92, 2]
[93, 12]
[94, 44]
[95, 10]
[96, 3]
[97, 18]
[98, 20]
[99, 97]
[100, 20]
[101, 21]
[102, 4]
[103, 6]
[104, 56]
[105, 2]
[106, 1]
[107, 2]
[108, 5]
************************************************
337
[1, 86]
[2, 83]
[3, 591]
[4, 67]
[5, 82]
[6, 3]
[7, 140]
[8, 23]
[9, 345]
[10, 23]
[11, 

...

[76, 15]
[77, 36]
[78, 27]
[79, 28]
[80, 3]
[81, 2]
[82, 31]
[83, 17]
[84, 6]
[85, 29]
[86, 21]
[87, 41]
[88, 41]
[89, 17]
[90, 8]
[91, 6]
[92, 259]
[93, 12]
[94, 12]
[95, 45]
[96, 37]
[97, 4]
[98, 15]
[99, 11]
[100, 190]
[101, 31]
[102, 9]
[103, 16]
[104, 36]
[105, 18]
************************************************
354
[1, 106]
[2, 586]
[3, 302]
[4, 240]
[5, 10]
[6, 234]
[7, 133]
[8, 8]
[9, 125]
[10, 64]
[11, 57]
[12, 26]
[13, 47]
[14, 121]
[15, 777]
[16, 0]
[17, 414]
[18, 155]
[19, 453]
[20, 58]
[21, 7]
[22, 367]
[23, 173]
[24, 177]
[25, 4]
[26, 556]
[27, 6]
[28, 35]
[29, 4]
[30, 61]
[31, 21]
[32, 7]
[33, 62]
[34, 27]
[35, 60]
[36, 45]
[37, 45]
[38, 56]
[39, 143]
[40, 89]
[41, 144]
[42, 45]
[43, 0]
[44, 0]
[45, 302]
[46, 153]
[47, 262]
[48, 44]
[49, 149]
[50, 48]
[51, 290]
[52, 79]
[53, 1]
[54, 123]
[55, 193]
[56, 132]
[57, 9]
[58, 102]
[59, 49]
[60, 49]
[61, 32]
[62, 397]
[63, 74]
[64, 146]
[65, 72]
[66, 2]
[67, 189]
[68, 112]
[69, 40]
[70, 419]
[71, 1439]
[72, 108]
[73, 24]
[74, 

[85, 986]
[86, 961]
[87, 187]
[88, 903]
[89, 518]
[90, 295]
[91, 204]
[92, 0]
[93, 0]
[94, 0]
[95, 105]
[96, 45]
[97, 0]
************************************************
363
[1, 6]
[2, 5]
[3, 16]
[4, 4]
[5, 9]
[6, 5]
[7, 1]
[8, 148]
[9, 6]
[10, 1]
[11, 7]
[12, 0]
[13, 2]
[14, 7]
[15, 13]
[16, 5]
[17, 9]
[18, 2]
[19, 50]
[20, 0]
[21, 0]
[22, 7]
[23, 27]
[24, 21]
[25, 1]
[26, 14]
[27, 0]
[28, 162]
[29, 8]
[30, 0]
[31, 25]
[32, 1]
[33, 1]
[34, 5]
[35, 2]
[36, 8]
[37, 6]
[38, 2]
[39, 13]
[40, 3]
[41, 15]
[42, 0]
[43, 12]
[44, 18]
[45, 91]
[46, 5]
[47, 14]
[48, 28]
[49, 24]
[50, 0]
[51, 28]
[52, 12]
[53, 5]
[54, 0]
[55, 3]
[56, 2]
[57, 19]
[58, 8]
[59, 2]
[60, 12]
[61, 6]
[62, 0]
[63, 7]
[64, 10]
************************************************
364
[1, 13]
[2, 13]
[3, 48]
[4, 1]
[5, 21]
[6, 46]
[7, 12]
[8, 18]
[9, 209]
[10, 255]
[11, 12493]
[12, 4]
[13, 22]
[14, 27]
[15, 92]
[16, 7]
[17, 1075]
[18, 9]
[19, 17]
[20, 2081]
[21, 22]
[22, 723]
[23, 446]
[24, 297]
[25, 224]
[26, 124]
[27, 162]
[

[51, 98]
[52, 36]
[53, 22]
[54, 131]
[55, 95]
[56, 0]
[57, 20]
[58, 60]
[59, 61]
[60, 16]
[61, 22]
[62, 31]
[63, 52]
[64, 5]
[65, 0]
[66, 47]
[67, 47]
[68, 57]
[69, 97]
[70, 61]
[71, 12]
[72, 103]
[73, 48]
[74, 68]
[75, 11]
[76, 10]
[77, 0]
[78, 17]
[79, 19]
[80, 28]
[81, 0]
[82, 9]
[83, 20]
[84, 223]
[85, 46]
[86, 0]
[87, 76]
[88, 0]
[89, 24]
[90, 48]
[91, 56]
[92, 111]
[93, 13]
[94, 91]
[95, 159]
[96, 19]
[97, 157]
[98, 32]
[99, 42]
[100, 19]
************************************************
373
[1, 11]
[2, 3]
[3, 5]
[4, 8]
[5, 1]
[6, 5]
[7, 2]
[8, 2]
[9, 4]
[10, 2]
[11, 0]
[12, 1]
[13, 33]
[14, 8]
[15, 25]
[16, 10]
[17, 15]
[18, 9]
[19, 2]
[20, 1]
[21, 3]
[22, 0]
[23, 1]
[24, 22]
[25, 21]
[26, 6]
[27, 4]
[28, 22]
[29, 7]
[30, 6]
[31, 1]
[32, 5]
[33, 6]
[34, 2]
[35, 1]
[36, 2]
[37, 3]
[38, 3]
[39, 5]
[40, 20]
[41, 15]
************************************************
374
[1, 166]
[2, 27]
[3, 6]
[4, 5]
[5, 64]
[6, 3]
[7, 18]
[8, 10]
[9, 32]
[10, 2]
[11, 21]
[12, 4]
[13, 49]
[14, 50]
[15

[2, 15]
[3, 4]
[4, 3]
[5, 9]
[6, 4]
[7, 10]
[8, 33]
[9, 16]
[10, 1]
[11, 6]
[12, 16]
[13, 25]
[14, 12]
[15, 0]
[16, 0]
[17, 0]
[18, 12]
[19, 1]
[20, 4]
[21, 20]
[22, 5]
[23, 0]
[24, 5]
[25, 10]
[26, 3]
[27, 2]
[28, 12]
[29, 3]
[30, 13]
[31, 2]
[32, 4]
[33, 12]
[34, 22]
[35, 7]
[36, 5]
[37, 10]
[38, 3]
[39, 15]
[40, 9]
[41, 4]
[42, 7]
[43, 1]
[44, 25]
[45, 19]
[46, 2]
[47, 13]
[48, 9]
[49, 11]
[50, 4]
[51, 6]
[52, 40]
[53, 6]
[54, 9]
[55, 1]
[56, 26]
[57, 7]
[58, 248]
[59, 39]
[60, 19]
[61, 4]
[62, 36]
[63, 5]
[64, 23]
[65, 20]
[66, 2]
[67, 4]
[68, 1]
[69, 5]
[70, 13]
[71, 2]
[72, 4]
[73, 40]
[74, 7]
[75, 11]
[76, 7]
[77, 3]
[78, 8]
[79, 6]
[80, 4]
[81, 4]
[82, 4]
[83, 9]
[84, 11]
[85, 3]
[86, 7]
[87, 16]
[88, 4]
[89, 3]
[90, 24]
[91, 23]
[92, 5]
[93, 30]
[94, 17]
[95, 0]
[96, 2]
[97, 0]
[98, 7]
[99, 14]
[100, 5]
[101, 2]
[102, 11]
[103, 1]
[104, 4]
[105, 9]
[106, 10]
[107, 8]
************************************************
383
[1, 49]
[2, 131]
[3, 14]
[4, 45]
[5, 16]
[6, 54]
[7, 4]
[8

[87, 4]
[88, 9]
[89, 133]
[90, 5]
[91, 26]
[92, 2]
[93, 10]
[94, 8]
[95, 5]
[96, 6]
[97, 31]
[98, 15]
[99, 6]
[100, 0]
[101, 55]
[102, 0]
[103, 19]
************************************************
391
[1, 21]
[2, 69]
[3, 56]
[4, 50]
[5, 49]
[6, 5]
[7, 5]
[8, 19]
[9, 75]
[10, 88]
[11, 174]
[12, 145]
[13, 110]
[14, 1016]
[15, 88]
[16, 231]
[17, 69]
[18, 135]
[19, 12]
[20, 67]
[21, 37]
[22, 108]
[23, 1262]
[24, 8]
[25, 53]
[26, 85]
[27, 51]
[28, 25]
[29, 66]
[30, 115]
[31, 16]
[32, 86]
[33, 106]
[34, 12]
[35, 24]
[36, 16]
[37, 22]
[38, 149]
[39, 72]
[40, 0]
[41, 26]
[42, 233]
[43, 291]
[44, 152]
[45, 18]
[46, 12]
[47, 12]
[48, 65]
[49, 435]
[50, 20]
[51, 41]
[52, 58]
[53, 14]
[54, 14]
[55, 57]
[56, 157]
[57, 67]
[58, 15]
[59, 12]
[60, 13]
[61, 28]
[62, 68]
[63, 193]
[64, 37]
[65, 51]
[66, 91]
[67, 16]
[68, 38]
[69, 14]
[70, 19]
[71, 100]
[72, 45]
[73, 108]
[74, 189]
[75, 14]
[76, 13]
[77, 30]
[78, 120]
[79, 77]
[80, 43]
[81, 44]
[82, 285]
[83, 38]
[84, 181]
[85, 44]
[86, 0]
[87, 7]
[88, 3

In [124]:
def index_one_file(termlist):
    fileIndex = {}
    for index, word in enumerate(termlist):
        if word in fileIndex.keys():
            fileIndex[word].append(index)
        else:
            fileIndex[word] = [index]
    return fileIndex

def make_indices(termlists):
    total = {}
    for filename in termlists.keys():
        items = index_one_file(termlists[filename])
        total[filename] = items
    return total

def fullIndex(index2):
    total_index = {}
    for filename in index2.keys():
        for word in index2[filename].keys():
            if word in total_index.keys():
                if filename in total_index[word].keys():
                    total_index[word][filename].extend(index2[filename][word][:])
                else:
                    total_index[word][filename] = index2[filename][word]
            else:
                total_index[word] = {filename: index2[filename][word]}
    return total_index

In [153]:
with open('new_index14.txt', 'w', encoding='utf-8', errors='ignore', newline='') as fout:
    writer = csv.writer(fout)
    for query in index1.keys():
        for file in index1[query].keys():
            writer.writerow([file, index1[query][file]])

In [171]:
with open('num_of_urls1.txt', 'w', encoding='utf-8', errors='ignore', newline='') as fout:
    writer = csv.writer(fout)
    for url in urls_count.keys():
        writer.writerow([url, urls_count[url]])

In [191]:
index2 = {}
full_index = {}
for query in index1.keys():
    index2[query] = make_indices(index1[query])
    full_index[query] = fullIndex(index2[query])

## BM25 + извлечение признаков из документов
### score: 0.31005 / 0.32990

In [181]:
abbr = pd.read_csv('abbrs.txt', sep=';', header=None)
abbrs = {}
for i in range(len(abbr[0])):
    fname = abbr[0][i]
    sname = abbr[1][i]
    if fname in abbrs.keys():
        abbrs[fname] += ' ' + sname
    else:
        abbrs[fname] = sname

In [183]:
#use first index
def len_of_docs(Index):
    lens = {}
    for file in Index.keys():
        lens[file] = len(Index[file])
    return lens

In [195]:
import requests

def distance(a, b):
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n
    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)
    return current_row[n]

def find_synonyms(word):
    if len(word) == 0:
        return []
    #url = 'http://slova.zkir.ru/dict/'
    url = 'https://synonymonline.ru/'
    url += word[0].title() + '/' + word
    raw_html = requests.get(url)
    raw_html.encoding = 'utf-8'
    soup = BeautifulSoup(raw_html.text)
    syn_list = []
    for item in soup.find_all("span"):
        w = item.string
        if w != 'Меню' and w != None:
            syn_list.append(item.string)
    return syn_list

def eng_to_rus_translate(string):
    layout = dict(zip(map(ord, "qwertyuiop[]asdfghjkl;'zxcvbnm,./`"
                           'QWERTYUIOP{}ASDFGHJKL:"ZXCVBNM<>?~'),
                           "йцукенгшщзхъфывапролджэячсмитьбю.ё"
                           'ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ,Ё'))
    return string.translate(layout)

def find_abbr(word):
    url = 'http://www.sokr.ru/'
    url += word + '/'
    raw_html = requests.get(url)
    raw_html.encoding = 'utf-8'
    soup = BeautifulSoup(raw_html.text)
    abbr_list = []
    for item in soup.find_all("p",  class_="value"):
        if item.string != None:
            abbr_list.append(re.sub('\xa0', ' ', item.string))
    for item in soup.find_all("p",  class_="comment"):
        if item.string != None:
            abbr_list.append(re.sub('\xa0', ' ', item.string))
    return abbr_list

def word_query(full_index, word):
    if word in list(stopwords.words):
        return []
    if word in full_index.keys():
        return [filename for filename in full_index[word].keys()]
    else:
        return []

def simple_query(fullIndex, Index1, Index2, string):
    index1_to_query = Index1[string]
    index2_to_query = Index2[string]
    full_index_to_query = fullIndex[string]
    
    new_string = string
    for substr in abbrs.keys():
        if substr in string:
            new_string += substr
    string = re.sub('[ \t\r\n]+', ' ', new_string)
    rank = Rank(string, full_index_to_query, index1_to_query, index2_to_query)
    result = []
    
    for word in string.split():
        word = word.lower()
        word = get_lemmatizer().parse(word)[0].normal_form
        for w in full_index_to_query.keys():
            d = distance(word, w)
            if d <= 2 and d > 0:
                res = word_query(full_index_to_query, w)
                result += res
                for doc in res:
                    if doc in rank.words_to_docs.keys():
                        rank.words_to_docs[doc].append(w)
                    else:
                        rank.words_to_docs[doc] = [w]
                
        synonyms = find_synonyms(word)
        for syn in synonyms:
            for w in syn.split():
                w = get_lemmatizer().parse(w)[0].normal_form
                res = word_query(full_index_to_query, w)
                result += res
                for doc in res:
                    if doc in rank.words_to_docs.keys():
                        rank.words_to_docs[doc].append(w)
                    else:
                        rank.words_to_docs[doc] = [w]
                        
        res = word_query(full_index_to_query, word)
        result += res
        for doc in res:
            if doc in rank.words_to_docs.keys():
                rank.words_to_docs[doc].append(word)
            else:
                rank.words_to_docs[doc] = [word]
        
    if len(rank.words_to_docs.keys()) < 10:
        for word in string.split():
            word = word.lower()
            for w in full_index_to_query.keys():
                d = distance(word, w)
                if d <= 4 and d > 0:
                    res = word_query(full_index_to_query, w)
                    result += res
                    for doc in res:
                        if doc in rank.words_to_docs.keys():
                            rank.words_to_docs[doc].append(w)
                        else:
                            rank.words_to_docs[doc] = [w]
    return rank.search()

In [213]:
import numpy as np

class Rank:
    def __init__(self, query, full_index, index1, index2):
        self.index1 = index1
        self.index2 = index2
        self.frequency = {}
        self.locations = {}
        self.mindistance = {}
        self.words_to_docs = {}
        self.docs_lens = len_of_docs(index1)
        self.query = query
        self.inbound_count = {}
        self.tf_idfs = {}
        
    def normalize_scores(self, scores, smallIsBetter=0):
        vsmall = 0.00001
        if smallIsBetter:
            minscore = min(scores.values())
            return {u: float(minscore)/max(vsmall, l) for (u,l) in scores.items()}
        else:
            maxscore = max(scores.values())
            if maxscore == 0:
                maxscore = vsmall
                return {u: float(c)/maxscore for (u,c) in scores.items()}
        return scores
    
    def frequency_score(self):
        for filename in self.words_to_docs.keys():
            for word in self.words_to_docs[filename]:
                if filename not in self.frequency.keys():
                    self.frequency[filename] = 0
                if word in self.index2[filename].keys():
                    self.frequency[filename] += len(self.index2[filename][word])
        return self.normalize_scores(self.frequency)
                
    def location_score(self):
        for filename in self.words_to_docs.keys():
            words_list = []
            for word in self.words_to_docs[filename]:
                if word in self.index2[filename].keys():
                    words_list += self.index2[filename][word]
            self.locations[filename] = np.median(words_list)/self.docs_lens[filename]
        return self.normalize_scores(self.locations, smallIsBetter=1)
    
    def distance_score(self):
        if len(self.query) <= 1:
            return {file: 1.0 for file in self.words_to_docs.keys()}
        for filename in self.words_to_docs.keys():
            words_list = []
            for word in self.words_to_docs[filename]:
                words_list += self.index2[filename][word]
            words_list.sort()
            if len(words_list) > 1:
                res = sum([abs(words_list[i]-words_list[i-1]) for i in range(1, len(words_list))])
                self.mindistance[filename] = res/len(words_list)
            else:
                self.mindistance[filename] = 1.0
        return self.normalize_scores(self.mindistance, smallIsBetter=1)
    
    def link_score(self):
        inbound_count = {}
        for file in self.words_to_docs.keys():
            f = open(file, 'r', encoding='utf-8', errors='ignore').readline()
            url = re.sub('\n', '', f)
            inbound_count[file] = urls_count[url]
        return self.normalize_scores(inbound_count)
    
    def get_scored_list(self):
        total_scores = {doc: 0 for doc in self.words_to_docs.keys()}
        weight_functions = [(2.0, self.BM25_score()), (1.0, self.frequency_score()), (1.0, self.location_score()), (1.0, self.distance_score()), (1.0, self.link_score())]
        for (weight, scores) in weight_functions:
            for doc in total_scores:
                total_scores[doc] += weight * scores[doc]
        return total_scores
    
    def search(self):
        scores = self.get_scored_list()
        ranked_scores = [(score, doc) for (doc, score) in scores.items()]
        ranked_scores.sort()
        ranked_scores.reverse()
        if len(ranked_scores) >= 10:
            e = 10
        else:
            e = len(ranked_scores)
        return [r[1] for r in ranked_scores[0:e]]
    
    def BM25_score(self):
        k = 2
        b = 0.75
        self.tf_idfs = {doc: 0 for doc in self.words_to_docs.keys()}
        avg_len = np.mean(list(self.docs_lens.values()))
        for word in self.query.split():
            word = word.lower()
            word = get_lemmatizer().parse(word)[0].normal_form
            N = len(index1.keys())
            if word not in full_index.keys():
                continue
            n = len(full_index[word].keys())
            idf = np.log((N - n + 0.5) / (n + 0.5))
            if idf < 0.001:
                idf = 0.001
            for doc in self.words_to_docs.keys():
                all_words = self.index1[doc]
                doc_len = len(all_words)
                tf = all_words.count(word) / doc_len
                if doc in self.tf_idfs.keys():
                    self.tf_idfs[doc] += idf*tf*(k+1)/(tf + k*(1-b+b*doc_len/avg_len))
        return self.normalize_scores(self.tf_idfs)

In [5]:
def res_to_ids(res):
    result = []
    for doc in res:
        result.append(docs_id[all_files.index(doc)])
    return result

In [219]:
import random
import time

result = []
n = 0
with open('new_res1.txt', 'w', encoding='utf-8', errors='ignore', newline='') as out:
    writer = csv.writer(out)
    writer.writerow(['QueryId', 'DocumentId'])
    for q in queries['query']:
        n += 1
        while True:
            try:
                res = simple_query(full_index, index1, index2, q)
                break
            except requests.exceptions.RequestException:
                time.sleep(10)
                
        res = res_to_ids(res)
        print([n, len(res)])
        if len(res) >= 10:
            result += res[:10]
        else:
            result += res
            k = len(res)
            while k < 10:
                rdoc = random.choice(docs_to_query[q])
                rdoc = res_to_ids([rdoc])
                if not (rdoc in res):
                    k += 1
                    result.append(rdoc)
        for i in result[-10:]:
            writer.writerow([n, i])
                    

[1, 10]
[2, 10]
[3, 10]
[4, 10]
[5, 10]
[6, 10]
[7, 10]
[8, 10]
[9, 10]
[10, 10]
[11, 10]
[12, 10]
[13, 10]
[14, 10]
[15, 10]
[16, 10]
[17, 10]
[18, 10]
[19, 10]
[20, 10]
[21, 10]
[22, 10]
[23, 10]
[24, 10]
[25, 10]
[26, 10]
[27, 10]
[28, 10]
[29, 10]
[30, 10]
[31, 10]
[32, 10]
[33, 10]
[34, 10]
[35, 10]
[36, 10]
[37, 10]
[38, 10]
[39, 10]
[40, 10]
[41, 10]
[42, 10]
[43, 10]
[44, 10]
[45, 10]
[46, 10]
[47, 10]
[48, 10]
[49, 10]
[50, 10]
[51, 10]
[52, 10]
[53, 10]
[54, 10]
[55, 10]
[56, 10]
[57, 10]
[58, 10]
[59, 10]
[60, 10]
[61, 10]
[62, 10]
[63, 10]
[64, 10]
[65, 10]
[66, 10]
[67, 10]
[68, 10]
[69, 10]
[70, 10]
[71, 10]
[72, 10]
[73, 10]
[74, 10]
[75, 10]
[76, 10]
[77, 10]
[78, 10]
[79, 10]
[80, 10]
[81, 10]
[82, 10]
[83, 10]
[84, 10]
[85, 10]
[86, 10]
[87, 10]
[88, 10]
[89, 10]
[90, 10]
[91, 10]
[92, 10]
[93, 10]
[94, 10]
[95, 10]
[96, 10]
[97, 10]
[98, 10]
[99, 10]
[100, 10]
[101, 10]
[102, 10]
[103, 10]
[104, 10]
[105, 10]
[106, 10]
[107, 10]
[108, 10]
[109, 10]
[110, 10]
[111, 10

## Новый индекс: с разделением документа на части

In [240]:
def process_files(query):
    morph = pymorphy2.MorphAnalyzer()
    PATTERN = re.compile(r'[A-Za-zА-Яа-я0-9]+')    
    file_to_terms = {}
    filenames = docs_to_query[query]
    for file in filenames:
        PYMORPHY_CACHE = {}
        f = open(file, 'r', encoding='utf-8', errors='ignore').read()
        lines = list(f)
        html = "".join(lines[1:])
        
        soup = BeautifulSoup(html)
        [s.extract() for s in soup(['script', 'style', 'title', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
        body = soup.get_text('\n', True).lower()
        body = PATTERN.findall(body)
        body = ' '.join([morph.parse(word)[0].normal_form for word in body])
        
        soup = BeautifulSoup(html)
        title = ' '.join(e.get_text() for e in soup.find_all('title')).lower()
        title = PATTERN.findall(title)
        title = ' '.join([morph.parse(word)[0].normal_form for word in title])
        
        head = ' '.join([e.get_text() for e in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]).lower()
        head = PATTERN.findall(head)
        head = ' '.join([morph.parse(word)[0].normal_form for word in head])
        
        file_to_terms[file] = {}
        file_to_terms[file]['body'] = body
        file_to_terms[file]['title'] = title
        file_to_terms[file]['head'] = head
    with open('{}.txt'.format(query), 'w') as out:
        writer = csv.writer(out)
        for file in filenames:
            writer.writerow([file, file_to_terms[file]])
    return file_to_terms

In [6]:
import numpy as np
import bs4
from bs4 import BeautifulSoup
import re
import os
from tqdm import tqdm
import pymorphy2
from _collections import defaultdict

from multiprocessing.pool import ThreadPool as Pool
from threading import Lock

In [34]:
mutex = Lock()
n_processed = 0
new_index1 = {}

def func_wrapper(q):
    process_files(q) 
    with mutex:
        global n_processed
        n_processed += 1
        print(f"\r{n_processed} objects are processed...", end='', flush=True)
    
with Pool(processes=15) as pool:
    pool.map(func_wrapper, list(queries['query']))

399 objects are processed...

## BM25F с расширениями запросов
### score: 0.72044 / 0.74728

In [227]:
def doc_to_dict(query):
    q = query+'.txt'
    new_data = pd.read_csv(q, header=None, engine='python')
    new_data.columns = ['name', 'data']
    res = {}
    for i in range(len(new_data.data)):
        cur_doc = re.sub('[\'\{\},]','', new_data.data[i])
        cur_doc = cur_doc.split(':')
        cur_res = []
        for item in cur_doc[1:]:
            item = ' '.join(item.split()[:-1])
            cur_res.append(item)
        res[new_data.name[i]] = cur_res
    return res

In [239]:
queries1 = pd.read_csv('queries.numerate1.txt', sep="	", header=None)
queries1.columns = ['id', 'query']

In [221]:
abbrs = pd.read_csv('abbrs.txt', sep = ';', header=None)

In [231]:
from tqdm import tqdm
PATTERN = re.compile(r'[A-Za-zА-Яа-я0-9]+')
morph = pymorphy2.MorphAnalyzer()

class BM25F:
    def __init__(self):
        self.total_len = 0
        self.title_len = 0
        self.head_len = 0
        self.body_len = 0
        self.title_idf = defaultdict(float)
        self.head_idf = defaultdict(float)
        self.body_idf = defaultdict(float)
        for query in tqdm(list(queries['old_query'])):
            data = doc_to_dict(query)
            query_id = queries['id'][queries['old_query'] == query].values[0]
            for doc in ranged_docs_to_query_head[query_id]:
                self.total_len += 1
                body = PATTERN.findall(data[doc][0].lower())
                self.body_len += len(body)
                self.calc_tf(body, self.body_idf)
                
                title = PATTERN.findall(data[doc][1].lower())
                self.title_len += len(title)
                self.calc_tf(title, self.title_idf)
                
                head = PATTERN.findall(data[doc][2].lower())
                self.head_len += len(head)
                self.calc_tf(head, self.head_idf)
        self.calc_idf(self.body_idf)
        self.body_len /= self.total_len
        self.calc_idf(self.title_idf)
        self.title_len /= self.total_len
        self.calc_idf(self.head_idf)
        self.head_len /= self.total_len
        
    def calc_tf(self, data, idfs_dict):
        included_to_idfs_dict = set()
        for word in data:
            if word not in included_to_idfs_dict:
                idfs_dict[word] += 1
                included_to_idfs_dict.add(word)
                
    def calc_idf(self, idfs_dict):
        idf_sum = 0
        negative_idf = []
        for word in idfs_dict:
            idf = np.log((self.total_len - idfs_dict[word] + 0.5)/(idfs_dict[word] + 0.5))
            if idf < 0:
                negative_idf.append(word)
            else:
                idfs_dict[word] = idf
            idf_sum += idf
        avg_idf = idf_sum / (len(idfs_dict) + len(negative_idf))
        for word in negative_idf:
            idfs_dict[word] = 0.1 * avg_idf

    def Rank(self, query):
        query_id = queries['id'][queries['query'] == query].values[0]
        doc_score = {}
        for doc in ranged_docs_to_query_head[query_id]:
            doc_score[doc] = self.scoring(query, doc)
        doc_score = sorted(doc_score.items(), key=lambda x: x[1], reverse=True)
        return doc_score

    def scoring(self, query, doc):
        old_query = queries['old_query'][queries['query'] == query].values[0]
        data = doc_to_dict(old_query)
        body = PATTERN.findall(data[doc][0].lower())
        body_len = len(body)
        body_tf = defaultdict(float)
        for word in body:
            body_tf[word] += 1
        title = PATTERN.findall(data[doc][1].lower())
        title_len = len(title)
        title_tf = defaultdict(float)
        for word in title:
            title_tf[word] += 1
        head = PATTERN.findall(data[doc][2].lower())
        head_len = len(head)
        head_tf = defaultdict(float)
        for word in head:
            head_tf[word] += 1
        
        new_q = query
        for substr1, substr2 in zip(abbrs[0], abbrs[1]):
            if substr1 in query.split():
                new_q += ' ' + substr2
        query = PATTERN.findall(' '.join(morph.parse(word)[0].normal_form for word in new_q.split()))
        k=2.0
        b=0.75
        title_score = 0
        head_score = 0
        body_score = 0
        for word in query:
            #if word in list(stopwords.words):
            #    continue
            body_score += self.body_idf[word] * body_tf[word] * (k + 1) / (body_tf[word] + \
                                                                        k * (1 - b + b * body_len / self.body_len))
            title_score += self.title_idf[word] * title_tf[word] * (k + 1) / (title_tf[word] + \
                                                                        k * (1 - b + b * title_len / self.title_len))
            head_score += self.head_idf[word] * head_tf[word] * (self.k + 1) / (head_tf[word] + \
                                                                        k * (1 - b + b * head_len / self.head_len))
        return 3.0 * title_score + 2.0 * body_score + 1.0 * head_score

In [37]:
model = BM25F()


 17%|█████████████▊                                                                   | 68/399 [01:47<09:08,  1.66s/it]


 34%|███████████████████████████▍                                                    | 137/399 [03:43<08:34,  1.96s/it]


 51%|█████████████████████████████████████████                                       | 205/399 [05:40<04:23,  1.36s/it]


 69%|███████████████████████████████████████████████████████▏                        | 275/399 [07:42<02:16,  1.10s/it]


 86%|████████████████████████████████████████████████████████████████████▉           | 344/399 [09:38<01:11,  1.29s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [10:53<00:00,  1.32s/it]

In [233]:
Query = []
docsId = []
for query in tqdm(queries['query']):
    best = model.Rank(query)
    for item in best:
        Query.append(query)
        docsId.append(res_to_ids([item[0]])[0])






  0%|                                                                                          | 0/399 [00:00<?, ?it/s]




  0%|▏                                                                               | 1/399 [00:55<6:05:28, 55.10s/it]




  1%|▍                                                                               | 2/399 [01:39<5:42:24, 51.75s/it]




  1%|▌                                                                               | 3/399 [04:11<9:00:03, 81.83s/it]




  1%|▊                                                                               | 4/399 [04:21<6:36:46, 60.27s/it]




  1%|█                                                                               | 5/399 [06:06<8:05:47, 73.98s/it]




  2%|█▏                                                                              | 6/399 [06:20<6:06:29, 55.95s/it]




  2%|█▍                                                                              | 7/399 [06:33<4:40:50, 42.99s/it]




  2

 17%|████████████▋                                                                | 66/399 [1:06:17<2:54:48, 31.50s/it]




 17%|████████████▉                                                                | 67/399 [1:06:24<2:14:05, 24.23s/it]




 17%|█████████████                                                                | 68/399 [1:09:08<6:04:25, 66.06s/it]




 17%|█████████████▎                                                               | 69/399 [1:09:28<4:47:21, 52.25s/it]




 18%|█████████████▌                                                               | 70/399 [1:09:37<3:36:15, 39.44s/it]




 18%|█████████████▋                                                               | 71/399 [1:12:11<6:43:35, 73.83s/it]




 18%|█████████████▉                                                               | 72/399 [1:14:09<7:54:22, 87.04s/it]




 18%|██████████████                                                               | 73/399 [1:14:17<5:43:42, 63.26s/it]




 19%|███

 33%|█████████████████████████▏                                                  | 132/399 [2:12:42<4:43:12, 63.64s/it]




 33%|█████████████████████████▎                                                  | 133/399 [2:12:48<3:25:21, 46.32s/it]




 34%|█████████████████████████▌                                                  | 134/399 [2:13:30<3:18:37, 44.97s/it]




 34%|█████████████████████████▋                                                  | 135/399 [2:16:18<6:00:26, 81.92s/it]




 34%|█████████████████████████▉                                                  | 136/399 [2:16:26<4:22:07, 59.80s/it]




 34%|██████████████████████████                                                  | 137/399 [2:16:33<3:11:35, 43.88s/it]




 35%|██████████████████████████▎                                                 | 138/399 [2:16:37<2:18:18, 31.80s/it]




 35%|██████████████████████████▍                                                 | 139/399 [2:17:55<3:17:44, 45.63s/it]




 35%|███

 50%|█████████████████████████████████████▋                                      | 198/399 [3:18:41<4:26:14, 79.48s/it]




 50%|█████████████████████████████████████▍                                     | 199/399 [3:21:48<6:12:48, 111.84s/it]




 50%|█████████████████████████████████████▌                                     | 200/399 [3:24:50<7:20:31, 132.82s/it]




 50%|██████████████████████████████████████▎                                     | 201/399 [3:24:59<5:16:03, 95.78s/it]




 51%|██████████████████████████████████████▍                                     | 202/399 [3:25:11<3:51:26, 70.49s/it]




 51%|██████████████████████████████████████▋                                     | 203/399 [3:26:44<4:12:09, 77.19s/it]




 51%|██████████████████████████████████████▊                                     | 204/399 [3:26:53<3:04:28, 56.76s/it]




 51%|███████████████████████████████████████                                     | 205/399 [3:27:51<3:04:41, 57.12s/it]




 52%|███

 66%|██████████████████████████████████████████████████▎                         | 264/399 [4:41:54<2:34:41, 68.75s/it]




 66%|██████████████████████████████████████████████████▍                         | 265/399 [4:42:48<2:23:58, 64.47s/it]




 67%|██████████████████████████████████████████████████▋                         | 266/399 [4:45:46<3:38:18, 98.49s/it]




 67%|██████████████████████████████████████████████████▊                         | 267/399 [4:45:47<2:32:12, 69.18s/it]




 67%|███████████████████████████████████████████████████                         | 268/399 [4:45:50<1:47:34, 49.27s/it]




 67%|███████████████████████████████████████████████████▏                        | 269/399 [4:47:11<2:07:32, 58.86s/it]




 68%|███████████████████████████████████████████████████▍                        | 270/399 [4:47:20<1:34:09, 43.80s/it]




 68%|███████████████████████████████████████████████████▌                        | 271/399 [4:47:56<1:28:42, 41.58s/it]




 68%|███

 83%|██████████████████████████████████████████████████████████████▊             | 330/399 [5:51:38<1:25:48, 74.61s/it]




 83%|███████████████████████████████████████████████████████████████             | 331/399 [5:52:56<1:25:42, 75.63s/it]




 83%|███████████████████████████████████████████████████████████████▏            | 332/399 [5:54:09<1:23:31, 74.80s/it]




 83%|███████████████████████████████████████████████████████████████▍            | 333/399 [5:54:39<1:07:29, 61.36s/it]




 84%|█████████████████████████████████████████████████████████████████▎            | 334/399 [5:55:06<55:06, 50.87s/it]




 84%|█████████████████████████████████████████████████████████████████▍            | 335/399 [5:55:40<48:59, 45.94s/it]




 84%|█████████████████████████████████████████████████████████████████▋            | 336/399 [5:57:01<59:12, 56.39s/it]




 84%|█████████████████████████████████████████████████████████████████▉            | 337/399 [5:57:27<48:53, 47.32s/it]




 85%|███

 99%|█████████████████████████████████████████████████████████████████████████████▍| 396/399 [6:44:42<03:00, 60.04s/it]




 99%|█████████████████████████████████████████████████████████████████████████████▌| 397/399 [6:44:46<01:26, 43.35s/it]




100%|█████████████████████████████████████████████████████████████████████████████▊| 398/399 [6:45:41<00:46, 46.63s/it]




100%|██████████████████████████████████████████████████████████████████████████████| 399/399 [6:46:10<00:00, 41.41s/it]

In [234]:
result = pd.DataFrame({'QueryId' : Query, 'DocumentId': docsId})

In [236]:
res = []
for query in result_df['QueryId']:
    res.append(queries['id'][queries['query'] == query].values[0])
result['QueryId'] = res

In [237]:
result[:10]

,QueryId,DocumentId
0,1,78
1,1,28
2,1,44
3,1,51
4,1,74
5,1,53
6,1,58
7,1,83
8,1,7
9,1,39


In [238]:
result.to_csv('result5.txt', index=False)

## BM25F -> LDA. Используя результаты result из предыдущего пункта.
### score: 0.69041 / 0.72611

In [48]:
import string
import gensim
import collections
from scipy.spatial import distance

In [94]:
import nltk
nltk.download('stopwords')
stop_words_ru = stopwords.words('russian')
stop_words_en = stopwords.words('english')
stop_words = stop_words_ru + stop_words_en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
def doc_to_dict(query):
    q = query+'.txt'
    new_data = pd.read_csv(q, header=None, engine='python')
    new_data.columns = ['name', 'data']
    docs_ids = res_to_ids(new_data.name.values)
    new_data.name = docs_ids
    res = {}
    for i in range(len(new_data.data)):
        cur_doc = re.sub('[\'\{\},]','', new_data.data[i])
        cur_doc = cur_doc.split(':')
        cur_res = []
        for item in cur_doc[1:]:
            item = ' '.join(item.split()[:-1])
            cur_res.append(item)
        res[new_data.name[i]] = cur_res
    return res

In [162]:
ranged_docs_to_query_head = {}
ranged_docs_to_query = {}
for i in range(399):
    ranged_docs_to_query_head[i+1] = result.DocumentId[result_df['QueryId'] == i+1].values[:10]
    ranged_docs_to_query[i+1] = result.DocumentId[result_df['QueryId'] == i+1].values

In [192]:
def LDA_one_query(texts):
    dictionary = gensim.corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10)
    topicsbook = np.zeros((len(corpus), lda.num_topics))
    for di,doc in enumerate(corpus):
        doc_top = lda[doc]
        for ti,tv in doc_top:
            topicsbook[di,ti] += tv
    weights = topicsbook.sum(0)
    distances = distance.squareform(distance.pdist(topicsbook))
    large = distances.max() + 1
    for i in range(len(distances)):
        distances[i, i] = large
    return (corpus, lda,weights,distances)

In [193]:
def LDA_Rank():
    lda_score = {}
    for query in tqdm(queries['old_query']):
        data = doc_to_dict(query)
        query_text = []
        query_id = queries['id'][queries['old_query'] == query].values[0]
        lda_score[query_id] = {}
        for doc in ranged_docs_to_query_head[query_id]:
            row_str = data[doc][0].split()
            query_text.append([word for word in row_str if word not in stop_words])
        corpus,lda,weights,distances = LDA_one_query(query_text)
        dictionary = gensim.corpora.Dictionary([query.split()])
        query_topic = sorted(lda.get_document_topics(dictionary.doc2bow(query.split())), key=lambda x: x[1], reverse=True)[0][0]
        for doc in ranged_docs_to_query_head[query_id]:
            row_str = data[doc][0].split()
            cur_text = [word for word in row_str if word not in stop_words]
            dictionary = gensim.corpora.Dictionary([cur_text])
            corpus = dictionary.doc2bow(cur_text)
            lda_score[query_id][doc] = 0
            for (topic, val) in lda.get_document_topics(corpus):
                if query_topic == topic:
                    lda_score[query_id][doc] = val
    return lda_score

In [194]:
import warnings
warnings.filterwarnings('ignore')
lda_score = LDA_Rank()






  0%|                                                                                          | 0/399 [00:00<?, ?it/s]




  0%|▏                                                                               | 1/399 [00:28<3:09:22, 28.55s/it]




  1%|▍                                                                               | 2/399 [00:51<2:58:32, 26.98s/it]




  1%|▌                                                                               | 3/399 [01:48<3:56:45, 35.87s/it]




  1%|▊                                                                               | 4/399 [01:57<3:03:28, 27.87s/it]




  1%|█                                                                               | 5/399 [03:00<4:10:55, 38.21s/it]




  2%|█▏                                                                              | 6/399 [03:11<3:17:58, 30.22s/it]




  2%|█▍                                                                              | 7/399 [03:22<2:40:25, 24.55s/it]




  2

 17%|█████████████                                                                  | 66/399 [33:16<1:32:45, 16.71s/it]




 17%|█████████████▎                                                                 | 67/399 [33:24<1:18:55, 14.26s/it]




 17%|█████████████▍                                                                 | 68/399 [34:53<3:22:07, 36.64s/it]




 17%|█████████████▋                                                                 | 69/399 [35:08<2:45:07, 30.02s/it]




 18%|█████████████▊                                                                 | 70/399 [35:17<2:09:54, 23.69s/it]




 18%|██████████████                                                                 | 71/399 [36:15<3:05:57, 34.02s/it]




 18%|██████████████▎                                                                | 72/399 [37:02<3:26:47, 37.94s/it]




 18%|██████████████▍                                                                | 73/399 [37:10<2:36:41, 28.84s/it]




 19%|███

 33%|█████████████████████████▏                                                  | 132/399 [1:07:22<2:27:17, 33.10s/it]




 33%|█████████████████████████▎                                                  | 133/399 [1:07:29<1:52:29, 25.37s/it]




 34%|█████████████████████████▌                                                  | 134/399 [1:07:55<1:52:52, 25.56s/it]




 34%|█████████████████████████▋                                                  | 135/399 [1:09:10<2:57:21, 40.31s/it]




 34%|█████████████████████████▉                                                  | 136/399 [1:09:20<2:16:06, 31.05s/it]




 34%|██████████████████████████                                                  | 137/399 [1:09:27<1:44:09, 23.85s/it]




 35%|██████████████████████████▎                                                 | 138/399 [1:09:32<1:19:15, 18.22s/it]




 35%|██████████████████████████▍                                                 | 139/399 [1:10:19<1:56:14, 26.83s/it]




 35%|███

 50%|█████████████████████████████████████▋                                      | 198/399 [1:42:21<2:03:44, 36.94s/it]




 50%|█████████████████████████████████████▉                                      | 199/399 [1:43:37<2:41:37, 48.49s/it]




 50%|██████████████████████████████████████                                      | 200/399 [1:44:45<3:00:42, 54.48s/it]




 50%|██████████████████████████████████████▎                                     | 201/399 [1:44:55<2:15:33, 41.08s/it]




 51%|██████████████████████████████████████▍                                     | 202/399 [1:45:06<1:45:33, 32.15s/it]




 51%|██████████████████████████████████████▋                                     | 203/399 [1:45:55<2:01:14, 37.11s/it]




 51%|██████████████████████████████████████▊                                     | 204/399 [1:46:03<1:31:57, 28.30s/it]




 51%|███████████████████████████████████████                                     | 205/399 [1:46:33<1:32:49, 28.71s/it]




 52%|███

 66%|██████████████████████████████████████████████████▎                         | 264/399 [2:20:55<1:13:19, 32.59s/it]




 66%|██████████████████████████████████████████████████▍                         | 265/399 [2:21:23<1:09:27, 31.10s/it]




 67%|██████████████████████████████████████████████████▋                         | 266/399 [2:22:43<1:41:43, 45.89s/it]




 67%|██████████████████████████████████████████████████▊                         | 267/399 [2:22:45<1:11:58, 32.71s/it]




 67%|████████████████████████████████████████████████████▍                         | 268/399 [2:22:51<53:26, 24.48s/it]




 67%|███████████████████████████████████████████████████▏                        | 269/399 [2:23:28<1:01:29, 28.38s/it]




 68%|████████████████████████████████████████████████████▊                         | 270/399 [2:23:36<47:47, 22.23s/it]




 68%|████████████████████████████████████████████████████▉                         | 271/399 [2:23:55<45:35, 21.37s/it]




 68%|███

 83%|████████████████████████████████████████████████████████████████▌             | 330/399 [2:55:10<45:51, 39.88s/it]




 83%|████████████████████████████████████████████████████████████████▋             | 331/399 [2:55:54<46:35, 41.12s/it]




 83%|████████████████████████████████████████████████████████████████▉             | 332/399 [2:56:31<44:29, 39.84s/it]




 83%|█████████████████████████████████████████████████████████████████             | 333/399 [2:56:53<38:09, 34.69s/it]




 84%|█████████████████████████████████████████████████████████████████▎            | 334/399 [2:57:21<35:15, 32.54s/it]




 84%|█████████████████████████████████████████████████████████████████▍            | 335/399 [2:57:44<31:37, 29.65s/it]




 84%|█████████████████████████████████████████████████████████████████▋            | 336/399 [2:58:33<37:15, 35.49s/it]




 84%|█████████████████████████████████████████████████████████████████▉            | 337/399 [2:58:49<30:43, 29.73s/it]




 85%|███

 99%|█████████████████████████████████████████████████████████████████████████████▍| 396/399 [3:30:11<01:55, 38.53s/it]




 99%|█████████████████████████████████████████████████████████████████████████████▌| 397/399 [3:30:19<00:58, 29.16s/it]




100%|█████████████████████████████████████████████████████████████████████████████▊| 398/399 [3:31:07<00:34, 34.80s/it]




100%|██████████████████████████████████████████████████████████████████████████████| 399/399 [3:31:32<00:00, 31.89s/it]

In [195]:
QueryId = []
DocumentId = []
for q_id in lda_score.keys():
    sorted_score = sorted(lda_score[q_id].items(), key = lambda x: x[1], reverse=True)
    for doc_id, _ in sorted_score:
        QueryId.append(q_id)
        DocumentId.append(doc_id)

In [196]:
result1 = pd.DataFrame({'QueryId' : QueryId, 'DocumentId': DocumentId})

In [197]:
result1.to_csv('result2.txt', index=False)